In [1]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import torch
from bpemb import BPEmb
from tqdm import tqdm
import nltk


In [2]:
# import training data
dft_eng = pd.read_csv('../../data/dft_eng.csv')
dft_jap = pd.read_csv('../../data/dft_jap.csv')
dft_fin = pd.read_csv('../../data/dft_fin.csv')

# import validation data
dfv_eng = pd.read_csv('../../data/dfv_eng.csv')
dfv_jap = pd.read_csv('../../data/dfv_jap.csv')
dfv_fin = pd.read_csv('../../data/dfv_fin.csv')

#import word count
word_count = pd.read_csv('../../data/question_word_count.csv')

dft_eng.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,answer_start,answer_text,question_text_tokenized,document_plaintext_tokenized,answer_text_tokenized,labels
0,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': array([159]), 'answer_text': ...",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,[159],['1920s'],"['when', 'was', 'quantum', 'field', 'theory', ...","['quantum', 'field', 'theory', 'naturally', 'b...",['1920s'],1
1,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': array([610]), 'answer_text': ...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,[610],['Sully Prudhomme'],"['who', 'was', 'the', 'first', 'nobel', 'prize...","['the', 'nobel', 'prize', 'in', 'literature', ...","['sully', 'prudhomme']",1
2,When is the dialectical method used?,Dialectic,english,"{'answer_start': array([129]), 'answer_text': ...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,[129],['discourse between two or more people holding...,"['when', 'is', 'the', 'dialectical', 'method',...","['dialectic', 'or', 'dialectics', '(', 'greek'...","['discourse', 'between', 'two', 'or', 'more', ...",1
3,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': array([88]), 'answer_text': a...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,[88],['Sejong the Great'],"['who', 'invented', 'hangul', '?']","['hangul', 'was', 'personally', 'created', 'an...","['sejong', 'the', 'great']",1
4,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': array([0]), 'answer_text': ar...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,[0],"['Grasshoppers are plant-eaters, with a few sp...","['what', 'do', 'grasshoppers', 'eat', '?']","['grasshoppers', 'are', 'plant-eaters', ',', '...","['grasshoppers', 'are', 'plant-eaters', ',', '...",1


In [3]:
# Load english model with 25k word-pieces
bpemb_en = BPEmb(lang='en', dim=100, vs=25000)

In [4]:
def get_bpemb_features(dataset, bpemb):
  # With bpemb we can tokenize and embed an entire document using .embed(x)
  X = [bpemb.embed(x).mean(0) for x in (dataset.document_plaintext)]
  y = dataset.labels
 
  return X,y

In [5]:
X_train,y_train = get_bpemb_features(dft_eng, bpemb_en)
X_test,y_test = get_bpemb_features(dfv_eng, bpemb_en)
lr_bpemb = LogisticRegression(penalty='l2', max_iter=1000, multi_class='multinomial')
lr_bpemb.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [6]:
preds_bpemb = lr_bpemb.predict(X_test)
preds_valid_bpemb = lr_bpemb.predict(X_test)

In [7]:
# BPEmb model 
report = classification_report(y_test, preds_bpemb, output_dict=True)
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.708061,0.656566,0.681342,495.000000
1,0.679849,0.729293,0.703704,495.000000
accuracy,0.692929,0.692929,0.692929,0.692929
macro avg,0.693955,0.692929,0.692523,990.000000
weighted avg,0.693955,0.692929,0.692523,990.000000


Text generation

In [15]:
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, output_hidden_states = True)

In [9]:
def generate_text(text, model, tokenizer, max_length=100):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def get_hidden_states(text, model, tokenizer):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    output = model(input_ids)
    return output.hidden_states

def generate_all(text, model, tokenizer, max_length, num_beams, no_reapeat_ngrams, temperature, top_k, top_p):
    input_ids = tokenizer.encode(text, return_tensors='pt')

    greedy_output = model.generate(input_ids, max_length=max_length, do_sample=False)
    beam_search_output = model.generate(input_ids, max_length=max_length, do_sample=True, num_beams=num_beams)
    n_grams_output = model.generate(input_ids, max_length=max_length, no_repeat_ngram_size=no_reapeat_ngrams, num_beams=num_beams)
    sample_output = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=temperature)
    top_k_otput = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=top_k)
    top_p_output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=top_k, top_p=top_p)

    output_lst = [greedy_output, beam_search_output, n_grams_output, sample_output, top_k_otput, top_p_output]
    decoded_samples = [tokenizer.decode(g[0], skip_special_tokens=True) for g in output_lst]

    return decoded_samples

def generate_top_p(text, model, tokenizer, max_length, top_p, top_k):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, do_sample=True,top_k=top_k, top_p=top_p)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_sample(text, model, tokenizer, max_length, temperature):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=temperature)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [10]:
samples = generate_top_p(dft_eng.question_text[0], model, tokenizer, 100, 0.9, 50)
print(samples)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


When was quantum field theory developed? Was it the effect of quantum field theory itself? How does quantum field theory explain the quantum field of quantum field theory?


Here we are in a state of quantum gravity. A quantum field of quantum gravity exists which cannot be measured and not measured, as opposed to the particles in the waveform. Therefore there are a few advantages to quantum field theory - that is, they provide quantum field theory (and thus can be demonstrated in the quantum field of quantum


In [11]:
#generate_top_p("The weather is", model, tokenizer, 50, 0.95, 50)
get_hidden_states("The weather is", model, tokenizer)[-1]


tensor([[[-0.0301,  0.3668,  0.0901,  ..., -0.2221,  0.1273, -0.1497],
         [ 0.5815,  0.2135,  0.1955,  ...,  0.4937,  0.0897, -0.2369],
         [ 0.2250,  0.5953, -0.3460,  ...,  0.3857, -0.0740,  0.1617]]],
       grad_fn=<ViewBackward0>)

In [12]:
input_ids = tokenizer.encode('I was meaning to', return_tensors='pt')

In [13]:
def get_sentence_perplexity(sentence, model, vocabulary, seq_len):
  states = (torch.zeros(lstm_layers, 1, lstm_dim).to(device),
              torch.zeros(lstm_layers, 1, lstm_dim).to(device))
  token_ids = [{'input_ids': bpemb_en.encode_ids(sentence)}]
  batch = collate_batch_bilstm(token_ids)
  loss_fn = torch.nn.CrossEntropyLoss()
  logits, states = model(batch[0].to(device), batch[1].to(device), states)

  target = batch[2].to(device)[:len(token_ids[0]['input_ids'])-1]
  loss = loss_fn(logits, target.reshape(-1))
  loss = loss.detach().cpu().numpy()
  return np.exp(loss)

In [18]:
from datasets import load_dataset

dataset = load_dataset("wikitext")

ModuleNotFoundError: No module named 'datasets'

In [23]:
from datasets import load_dataset

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to C:/Users/Hallgrimur/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


In [39]:
"\n\n".join(dft_eng["question_text"])
encodings
device = "cpu"


In [40]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over input tokens.
        # Multiply it with trg_len to get the summation instead of average.
        # We will take average over all the tokens to get the true average
        # in the last step of this example.
        neg_log_likelihood = outputs.loss * trg_len

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

 36%|███▋      | 205/562 [11:42<20:01,  3.37s/it]